In [1]:
import numpy as np
import random
import sys

In [2]:
with open('poems.txt','r', encoding='utf-8') as file:
    text=file.read()

print(text)

﻿Buffalo Bill’s
defunct
who used to
ride a watersmooth-silver
stallion
and break one two three four five pigeons just like that
Jesus

he was a handsome man
and what i want to know is
how do you like your blueeyed boy
Mister Death

Had I the heaven’s embroidered cloths,
Enwrought with golden and silver light,
The blue and the dim and the dark cloths
Of night and light and the half-light,
I would spread the cloths under your feet:
But I, being poor, have only my dreams;
I have spread my dreams under your feet;
Tread softly because you tread on my dreams.

He clasps the crag with crooked hands;
Close to the sun in lonely lands,
Ring’d with the azure world, he stands.

The wrinkled sea beneath him crawls;
He watches from his mountain walls,
And like a thunderbolt he falls.

Some say the world will end in fire,
Some say in ice.
From what I’ve tasted of desire
I hold with those who favor fire.
But if it had to perish twice,
I think I know enough of hate
To say that for destruction ice
Is al

In [3]:
unique_char = sorted(set(text))
char_to_index = dict((c,i) for i,c in enumerate(unique_char)) # converting character to numeric format
index_to_char = dict((i,c) for i,c in enumerate(unique_char)) # converting numberic back to character format

In [4]:
seq_length= 20 # how many characters are going to be used as features
step= 3 # how many characters are you shift
''' if seq='Hello World' with seq_len=5 and step =3 'Hello' is the
feature of len 5 and the next 5 features is 'lo Wo' because of shift 3 '''

sentences = [] # features
next_characters = [] # target variable
# if sentences='how are yo' next_characters = 'u'

Filling up the features and target values

In [5]:
for i in range(0, len(text) - seq_length,step):
    sentences.append(text[i:i+seq_length])
    next_characters.append(text[i+seq_length]) 

convert training data into numerical formal using numpy

In [6]:
x= np.zeros((len(sentences), seq_length, len(unique_char)), dtype=np.bool_)
y= np.zeros((len(sentences),len(unique_char)), dtype=np.bool_)

In [7]:
for i, sen in enumerate(sentences):
    for j, char in enumerate(sen):
        x[i,j, char_to_index[char]] = 1
    y[i, char_to_index[next_characters[i]]]=1

Training GRU

In [8]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, GRU, Activation
from keras.optimizers import RMSprop

In [9]:
model = Sequential([
    GRU(128, input_shape= (seq_length, len(unique_char))),
    Dense(len(unique_char), activation='softmax')
])

In [10]:
model.compile(loss='categorical_crossentropy', optimizer= 'rmsprop')

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 128)               69504     
                                                                 
 dense (Dense)               (None, 51)                6579      
                                                                 
Total params: 76,083
Trainable params: 76,083
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(x,y, batch_size=64, epochs=100)

Epoch 1/100
10/10 [==============================] - 17s 24ms/step - loss: 3.7547
Epoch 2/100
10/10 [==============================] - 0s 20ms/step - loss: 3.1956
Epoch 3/100
10/10 [==============================] - 0s 20ms/step - loss: 3.1259
Epoch 4/100
10/10 [==============================] - 0s 22ms/step - loss: 3.1077
Epoch 5/100
10/10 [==============================] - 0s 21ms/step - loss: 3.0965
Epoch 6/100
10/10 [==============================] - 0s 21ms/step - loss: 3.0593
Epoch 7/100
10/10 [==============================] - 0s 21ms/step - loss: 3.0449
Epoch 8/100
10/10 [==============================] - 0s 21ms/step - loss: 3.0286
Epoch 9/100
10/10 [==============================] - 0s 20ms/step - loss: 2.9914
Epoch 10/100
10/10 [==============================] - 0s 21ms/step - loss: 2.9816
Epoch 11/100
10/10 [==============================] - 0s 20ms/step - loss: 2.9748
Epoch 12/100
10/10 [==============================] - 0s 23ms/step - loss: 2.9387
Epoch 13/100
10/10 [====

In [13]:
model.save_weights('weights.hdf5')

In [14]:
def sample(preds,temp=1.0):
  preds=np.asarray(preds).astype('float64')
  preds=np.log(preds)/temp
  exp_pred=np.exp(preds)
  preds=exp_pred/np.sum(exp_pred)
  probs=np.random.multinomial(1,preds,1)
  return np.argmax(probs)

In [35]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

filepath= 'weights.hdf5'
checkpoint= ModelCheckpoint(filepath, monitor='loss', save_best_only=True, mode= 'min')
reduce= ReduceLROnPlateau(monitor='loss', factor= 0.2, patience=1, min_lr=0.0001)

In [57]:
from keras.callbacks import LambdaCallback

def generate_text(length=100, temperature=1.0):
    for temperature in [0.2,0.4,0.6,0.8,1.0]:
        model.load_weights('weights.hdf5')
        print("--------------------------Temperature = ",temperature,"--------------------------")
        start_index= random.randint(0, len(text)-seq_length-1)
        generated= ''
        sentence = text[start_index: start_index+seq_length]
        generated = sentence
        for i in range(length):
            x_pred= np.zeros((1,seq_length,len(unique_char)))
            for j, char in enumerate(sentence):
                x_pred[0,j,char_to_index[char]]=1
    
            predictions = model.predict(x_pred, verbose=0)[0]
            #predictions = np.asarray(predictions).astype('float64')
            next_index = sample(predictions, temperature)
            next_char = index_to_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
        print(generated)
    return generated

print_callback = LambdaCallback(on_epoch_end = generate_text)

In [58]:
callbacks=[print_callback, checkpoint, reduce]

In [59]:
print(generate_text(300,0.4))

--------------------------Temperature =  0.2 --------------------------
er, long I stood
And siour,
Th aly trer fir an ooreane wasth
 heaverean loreat he  acer

Some san bree as ffr the in an oureali ha  ay tre the sine  aolree ank lreals on  ceal;
Theaver ag tre ther ano lo ha some thaver an llerea soore wink e an an oureal oore I vond I lour an lleean soor the cald so le hav agoure wick 
--------------------------Temperature =  0.4 --------------------------
d trodden black.
Oh, divor fin an oor the cast 
in an oure li fee fire siolle an al orretthe angh
And wank ded se the in an  liureed io the inga I dice I dilo he aave  an an owe freals,on angr an llyee sire the bagh dedece than boure siol ,ed silr fe treald wank , d cant the cayth
Thong the thea soore disy a thur an  l
--------------------------Temperature =  0.6 --------------------------
etter claim,
Because wi the an an owrrrre fise and lo chat 
i  ane li an  lath
And wanln, disy t the wand le ane lreacs

Sothe annt I lfur fii